**DeapSECURE module 5: Cryptography for Privacy-Preserving Computation (part B: Homomorphic Encryption)**

# Hands-On Session 3: Paillier Encryption

Welcome to the DeapSECURE online training program!
This is a Jupyter notebook for the hands-on learning activities of the ["Cryptography for Privacy-Preserving Computation"](https://deapsecure.gitlab.io/deapsecure-lesson05-crypt/) module, Episode 5: ["Paillier Cryptosystem for Homomorphic Encryption"](https://deapsecure.gitlab.io/deapsecure-lesson05-crypt/21-paillier-he/index.html).
Please visit the [DeapSECURE](https://deapsecure.gitlab.io/) website to learn more about our training program.

In this session, we will learn a special encryption called **homomorphic encryption** which allows computation over encrypted quantities. We focus on the Paillier cryptosystem, which falls under the class of **partially homomorphic encryption**.

Our goal for this notebook is to gather some interesting observations on the Paillier cryptosystem, including how it can be used to perform arithmetic on encrypted numbers.
If you want to know more details (including the mathematical theory behind this cryptosystem), please see, e.g., https://en.wikipedia.org/wiki/Paillier_cryptosystem .

<a id="TOC"></a>
**Quick Links** (sections of this notebook):

* 1 [Setup](#sec-Setup)
* 2 [Key Generation](#sec-KeyGeneration)
* 3 [Saving and Loading Keypair](#sec-Save-load-keypair)
* 4 [Working with Encrypted Numbers](#sec-Encrypt-numbers)
* 5 [Timing the Homomorphic Operations](#sec-Timing)
* 6 [Encrypting Digital Images](#sec-Encrypt-images)

<a id="sec-Setup"></a>
## 1. Setup Instructions

If you are opening this notebook from Wahab cluster's OnDemand interface, you're all set.

If you see this notebook elsewhere and want to perform the exercises on Wahab cluster, please follow the steps outlined in our setup procedure.

1. Make sure you have [activated your HPC service](https://wiki.hpc.odu.edu/GettingStarted#getting-an-account).
2. Point your web browser to https://ondemand.wahab.hpc.odu.edu/ and sign in with your MIDAS ID and password.
3. Create a new Jupyter session using "legacy" Python suite, then create a new "Python3" notebook. (See [ODU HPC Wiki](https://wiki.hpc.odu.edu/en/ood-jupyter) for more detailed help.)
4. Get the necessary files using commands below within Jupyter:

       mkdir -p ~/CItraining/module-crypt
       cp -pr /shared/DeapSECURE/module-crypt/. ~/CItraining/module-crypt
       cd ~/CItraining/module-crypt

The file name of this notebook is `Paillier-session-1.ipynb`.

### 1.1 Reminder

* Throughout this notebook, `#TODO` is used as a placeholder where you need to fill in with something appropriate. 
* To run a code in a cell, press `Shift+Enter`.
* Use `ls` to view the contents of a directory.

### 1.2 Loading Python Libraries

<!--
Now we need to **import** the required libraries into this Jupyter notebook:`numpy`.
-->

**Important**: On Wahab HPC, software packages, including Python libraries, are managed and deployed via *environment modules*.
Before we can import the Python libraries in our current notebook, we have to load the corresponding environment modules.
We have setup a custom environment "DeapSECURE" which will load all the required libraries for this workshop. Please load "DeapSECURE" module.

* Load the modules above using the `module("load", "MODULE")` or `module("load", "MODULE1", "MODULE2", "MODULE n")` statement.
* Next, invoke `module("list")` to confirm that these modules are loaded.
* In this module, we have setup a custom environment "DeapSECURE" including all the required libraries. Please load "DeapSECURE" module. (You can also setup your [custom environment](https://wiki.hpc.odu.edu/Software/Python#install-additional-python-modules))

In [ ]:
"""Modify and uncomment statements below to load the required environment modules""";

#module("load", "#TODO")

In [ ]:
"""Confirm the loaded modules""";

#TODO

Now we can import the following standard Python libraries:
`json`, `re`, `os`, `time` and `numpy`.

In [ ]:
"""Uncomment, edit, and run code below to import libraries""";
#import #TODO

Next we import the Paillier encryption library for Python:

In [ ]:
# Paillier encryption library
import phe
from phe import paillier

We also load our own custom module to simplify programming with Paillier encryption data:

In [ ]:
# Our own auxiliary toolkit
import paillier_tools

<a id="sec-KeyGeneration"></a>
## 2. Key Generation

First, we learn how to generate a private-public keypair to encrypt and decrypt our numbers:
- the public key (pubkey) is used for encryption;
- the private key (privkey) is for decryption.

In [ ]:
pubkey, privkey = paillier.generate_paillier_keypair(n_length=2048)

The (public) key length determines the strength of the encryption.

**EXERCISE**: What's in a key? (*a.k.a. wear your curiosity hat!*)

Try to print the keys:

In [ ]:
pubkey

In [ ]:
privkey

These objects are opaque when printed "as is".

A cryptography key is essentially a set of one or more extremely large numbers, like prime numbers and the products thereof.
The basic premise of asymmetric cryptography is that it is computationally infeasible to reverse-engineer the private key from merely knowing the public key.

**The Private Key** --
In Paillier cryptosystem, the private key consists of two prime numbers, $p$ and $q$, which are stored as `privkey.p` and `privkey.q` attributes, respectively. These are extremely long integers, as you will see now:

In [ ]:
"""print privkey.p and privkey.q""";
#TODO

----------

> **OPTIONAL EXERCISE**: Count the number of digits in $p$ and $q$ above, respectively.
>
> For the curious in mind, here are some interesting exercises:
>
> 1. In decimal representation, how many digits long are $p$ and $q$?
>
> 2. In hexadecimal representation, how many digits long are $p$ and $q$?
>
> 3. Relate the last answer to the number of bits in $p$ and $q$. (Hint: Each hexadecimal digit has 4 bits.)
>
> *Hints*: There are several ways to go about finding the answer to the questions above. The following functions can be helpful: `str`, `hex`, `len`.

In [ ]:
"""Enter your response / experimentation here""";

> *(END OPTIONAL EXERCISE)*

-----------

**The Public Key** --
Formally, the public key consists of $n$ and $g$. By definition, 
- $n = p \times q$
- $g = n + 1$
so effectively the public key only has $n$.

**EXERCISE**: Print $n$ and $g$ and figure out how long these numbers are.

In particular, verify that the length of $n$ is as expected (2048 bits).

In [ ]:
# print n and g

#TODO

**QUESTIONS**:

1. Compare the lengths of the public-key numbers ($n$ and $g$) to $p$ and $q$. Do they make sense?

2. (Challenge) Can you estimate how much effort is required to factorize $n$ without knowing $p$ and $q$ a priori?

------

> **SIDEBAR: Base64 Representation**
>
> The [**base64** representation](https://en.wikipedia.org/wiki/Base64) is more compact than the decimal or hexadecimal representations and will be used to efficiently store long numbers such as $p$, $q$ and $n$. Here's an example:

In [ ]:
p_base64 = phe.util.int_to_base64(privkey.p)
print(p_base64)
print(len(p_base64))

In [ ]:
n_base64 = phe.util.int_to_base64(pubkey.n)
print(n_base64)
print(len(n_base64))

> Compare the length of this representation vs the length of `pubkey.n` in decimal representation. (Beware that base64 is an *encoding*, not an *encryption*, although the output looks cryptic to the eye.)
 
----------------

<a id="sec-Save-load-keypair"></a>
## 3. Saving and Loading Keypair

Every time `generate_paillier_keypair()` is called, a different keypair is generated, and the key values are random and unpredictable.
This randomness is essential to ensure security, i.e. to foil any attempt of guessing the generated keys.
Once generated, therefore, there has to be a way to save and restore a keypair in order to use them at a later time.

In this training module, we save our keys and encrypted values using a text-based JSON format that is *similar* to the JWK (JSON Web Key) format, which is used to transmit security keys over the Internet.
We prepare a custom Python module named `paillier_tools`, which primarily contains functions to save and load Paillier keypairs as well as encrypted numbers.
(Feel free to modify them to suit your own needs.)

> **NOTE**: `paillier_tools` is part of DeapSECURE software suite, primarily to support this training program.
> It is not currently available on public repo such as [PyPI](https://pypi.org/).
> 


### 3.1 Saving a Keypair

We use a function named `keypair_dump_jwk()` (defined in `paillier_tools`) to create a text representation of a pair of public/private keys. `keypair_dump_jwk()` returns two JSON strings---one for the public key and the other for the private key.
These JSON strings can be saved to text files and/or transmitted to other parties via Internet.

Use Python's `help(functionName)` or Jupyter's `functionName?` syntax to read the documentation on a particular function, e.g.

In [ ]:
help(paillier_tools.keypair_dump_jwk)

Now dump the keypair to the JSON strings, which can then be saved into text files:

In [ ]:
pubkey_jwk, privkey_jwk = paillier_tools.keypair_dump_jwk(pubkey, privkey)

In [ ]:
print(pubkey_jwk)

You see that the $n$ value is stored in base64 representation. Now, the keys can be written easily to a text file:

In [ ]:
# Save the keys for later uses
paillier_tools.write_file("phe_key.priv", privkey_jwk + "\n")
paillier_tools.write_file("phe_key.pub", pubkey_jwk + "\n")

### 3.2 Loading a Keypair

The `keypair_load_jwk()` function recreates a public/private keypair from a pair of JSON strings:

In [ ]:
# we first load the keys from file

privkey2_jwk = paillier_tools.read_file("phe_key.priv")
pubkey2_jwk = paillier_tools.read_file("phe_key.pub")

In [ ]:
# then recreate the key objects from the data:
pubkey2, privkey2 = paillier_tools.keypair_load_jwk(pubkey2_jwk, privkey2_jwk)

There are additional functions in `paillier_tools` which allow you to load only one key (either private or public).
Please look at the module's source code if you are interested.

**EXERCISE**: Please print the contents of the keys and make sure that they are identical to the ones generated above.

<a id="sec-Encrypt-numbers"></a>
## 4. Working with Encrypted Numbers

The keys generated above will unlock the fun part below, where we will encrypt numbers and operate on them without encrypting them until the final result!

The `phe.paillier` module can encrypt both integers and real numbers (floating point). The encryption will cryptically transform a plaintext into an integer is extremely large.

Let us begin with encrypting two numbers `m1` and `m2` below:

In [ ]:
m1 = 3.14159
m2 = 10.01

> It is advisable not to use numbers that are extreme in the magnitude (for example, $10^{100}$ [ten raised to the one-hundredth power] or beyond) because the extreme values can cause overflow in the representation of the ciphertext.

### Encrypting numbers

Use the public key to encrypt the numbers:

In [ ]:
"""Use `pubkey.encrypt()` function to encrypt m1 and m2:""";

# M1 = pubkey.encrypt(#TODO)
# M2 = pubkey.encrypt(#TODO)

> **CONVENTION**: We use variables with lowercase letters to denote unencrypted numbers (plaintexts), while those in uppercase letters denote encrypted numbers (ciphertexts).

**QUESTION**: 
What do the encrypted numbers look like?

In [ ]:
"""Print M1 and M2 here""";
#TODO

The encrypted value can be inspected by calling the `ciphertext()` method:

In [ ]:
"""Use `M1.ciphertext()` to print the encrypted value of m1.
Do the same with M2.""";
#TODO

**QUESTIONS**:
- Can you make sense of the numbers?
- How long is the ciphertext? Compare that to the private and public key numbers.
- How easy it is to reverse the original number without the knowledge of the private key?

### Encrypted Arithmetic

Using Paillier encryption scheme, the following operations can be done on the ciphertext (e.g. `M1`, `M2`, …):

- Addition of a ciphertext with another plaintext: M1 + n1 -> M2

- Multiplication of a ciphertext with another plaintext: M1 * n1 -> M3

- Addition on two ciphertexts: M1 + M4 -> M5

But multiplication of two ciphertexts are disallowed.

In [ ]:
"""Try addition on two or ciphertexts:
let `M3` be the sum of M1 and M2 and print the contents of M3.""";

#M3 = #TODO

**QUESTION**:
Does `M3.ciphertext()` equal `M1.ciphertext() + M2.ciphertext()`?

The results of the mathematical operations M3 are also encrypted numbers which are jibberish to those without the decryption key.

### Decrypting Numbers

Let us now **decrypt** the numbers, and check the result of our mysterious arithmetic above with the result of plaintext arithmetics.

In [ ]:
"""Use the `privkey.decrypt()` function to unveil the result:""";
# privkey.decrypt(#TODO)

**EXERCISES**:

- Decrypt `M1` and `M2` and print the result
- Add the original `m1` and `m2` and print the result

Does everything make sense now?

**EXERCISE**: Let's play around with some numbers and make sure you are comfortable with the mathematical operations!
Here are some examples---feel free to think of more:

* `M1 * 2`
* `2 * M1`
* `M1 / 2`
* `M1 * 2 + M2 / 2`

**EXERCISE**: Can we subtract two encrypted numbers? What would be the result? Verify against the arithmetic of the corresponding plaintexts!

**EXERCISE**: Can we multiply two encrypted texts?
Uncomment and run the following code to find out:

In [ ]:
#M1 * M2

### Saving and Loading Numbers

The `paillier_tools` module has a few functions to load and save encrypted numbers.
They are designed to work with one- and two-dimensional arrays:

* `envec_dump_json`: Dumps a vector (one-dimensional array) of numbers to a JSON string;
* `envec_load_json`: Loads a vector of encrypted numbers from a JSON string;
* `enimg_dump_json`: Dumps a two-dimensional array of encrypted numbers to a JSON string;
* `enimg_load_json`: Loads a two-dimensional array of encrypted numbers from a JSON string.

> **What's in the name?** The `envec` prefix means ***en***crypted ***vec***tor, where the `enimg` stands for ***enc***rypted ***im***a***g***e.

We will not have hands-on examples in this notebook for these functions.
In your hands-on directory, `~/CItraining/module-crypt/Paillier`, please find several Python scripts which serve as examples on using these functions.

### Fascinating Factoids

**EXERCISE**: Encrypt the same number twice (saving them to two different variables). Are the ciphertexts identical?

<a id="sec-Timing"></a>
## 5. Timing the Homomorphic Operations

Homomorphic encryption essentially transforms plain numbers into an extremely long integers that hide the actual values.
Given the length of the integers, we do not expect that these operations would be very fast.
But, **how fast or slow are the operations in homomorphic encryption (i.e. encryption, decryption, arithmetic)**?

In this section, you will measure the time taken to generate a keypair, encrypt numbers, decrypt numbers, add/multiply numbers.
To amplify the required time, we will perform the said computation 100 times each in the exercises below.


### Timing: Key Generation

**EXERCISE**: Generate 100 keypairs and note the amount of time taken.

*HINTS:* There are several ways to do this.

1. One way is to use a pair of `time.time()` function calls, before and after the section of the code that we want to time.

2. In Jupyter, we can use the `%time` magic to measure the time needed to run a line of Python code (specified following the `%time` keyword).
If code being timed has multiple lines, simply use `%%time%` instead.

Here is one example solution, using the *list comprehension* syntax:

In [ ]:
print("2048-bit key generation (100 keys)")
%time _tmp = [ paillier.generate_paillier_keypair(n_length=2048) for x in range(100) ]

Try at least another way to measure the 100 key generations; observe if the wall time is very close.

### Timing: Encryption

**EXERCISE**: Measure the amount of time required to encrypt 100 numbers.

First we use the `numpy.random.random` function to generate the plaintext numbers.
We create two sets of random numbers (`xx1` and `xx2`) that we will use later:

In [ ]:
xx1 = numpy.random.random((100,))
xx2 = numpy.random.random((100,))

In [ ]:
xx1[:10]

In [ ]:
xx2[:10]

Use the same approach as the above to encrypt the arrays and measure the total time required.

*HINTS:*

* Name the encrypted arrays `XX1` and `XX2`.
* The `pubkey.encrypt` function only takes one number at a time--it can't encrypt an array. What should you use to encrypt the numbers
* List comprehension makes the code short and easy to read.

### Timing: Homomorphic Arithmetic

**EXERCISES**:
Time the following operations:

* Addition of two HE numbers: *e.g.* `X1 + X2`
* Addition of one HE number and one plain number: *e.g.* `X1 + x3`
* Multiplication of one HE number and one plain number: *e.g.* `X1 * x3`
* Subtraction of two HE numbers: *e.g.* `X1 - X2`

Please be creative! Expand the list of operations you want to try!

Similar to the timing above, perform each operation 100 times and measure the total time taken.

> *HINTS:*
>
> * Use the arrays `XX1`, `XX2` to make up the numbers to work with.
> * You can create more array(s) as necessary.
> * Use loop or list comprehension as you see fit.

Here's an example statement to get started--but don't get stuck and think that this is the only way!

```python
%time XX1_2 = [ X1 + X2 for (X1,X2) in zip(XX1,XX2) ]
```

(PLACEHOLDER)

* Addition of one HE number and one plain number (in two different orders)

(PLACEHOLDER)

* Multiplication of one HE number and one plain number (in two different orders)

### Timing: Decryption

**EXERCISE**: Measure the amount of time required to decrypt 100 numbers (e.g. convert back numbers in `XX1` to a new array/list named `xx1_decrypt`).

### Timing Summary

**EXERCISE**: In this section, create a summary table of the various operations you tried so far.
Discuss the findings with your group.

Summary of Wahab's (2.4 GHz Intel Xeon 6148, 1 core) timing of computations on PHE operations (100 operations, 2048 bit key):

* Key generation: ___ secs
* Encryption: ___  secs
* Decryption: ___  secs
* Arithmetic (M1 + M2): ___  secs
* Arithmetic (M1 + m2): ___  secs
* Arithmetic (m1 + M2): ___  secs
* Arithmetic (M1 * m2): ___  secs

What are the implications of these timings?


> *HINTS:*
>
> * Add more entries to the list above for the additional arithmetic operations you also tried earlier.

### Additional Exercises (Optional)

These exercises will help you understand the computational complexity (intensity) of Paillier cryptosystem.

1. Decrease the security level of the key to 1024 bits. How much computing time do we save? (Our default key size is 2048 bits.)

2. Increase the security level of the key to 3072 bits. How much more computing time do we require?

3. How much time does AES encryption take for 100 numbers? (Remember: AES encryption & decryption operations take roughly the same amount of time.

### Scripting Exercises

These exercises are critical to processing large amounts of data.

1. Create a script that will encrypt a set of 10 numbers (make your own numbers, can be hardcoded inside the script for simplicity) and write the encrypted data to a file (say, `encrypted.phe`).

2. Create a script that will read in encrypted numbers `encrypted.phe` and decrypt them, and print the output to terminal.

> *HINTS:* To write/read encrypted numbers to disk, look at the helper functions available in `paillier_tools.py` and use them before devising your own function(s).

<a id="sec-Encrypt-images"></a>
## 6. Encrypting Digital Images

The last part of this notebook is also the ultimate goal of this workshop:
**Creating a program to encrypt digital images using Paillier cryptosystem**.

> ### Example Use Case: Remote Medical Diagnosis
>
> Machine learning (ML) and artificial intelligence (AI) have become a dominant part of society, including medicine.
> However, providing medical imaging data to remote parties for diagnosis is a difficult task to do, mainly due to strict laws and regulations, such as [HIPAA](https://www.cdc.gov/phlp/publications/topic/hipaa.html), governing the use and sharing of such kind of data.
> Encrypting sensitive medical images for ML/AI-assisted diagnosis is one way to prevent unauthorized diclosure of the data while still benefiting from ML/AI technologies developed by third parties.

As the ultimate part of this workshop, let us consider a program that encrypts a grayscale image.
A straightforward approach is to use Paillier library to encrypt the image, pixel-by-pixel.
This will allow the image to be secure processed or analyzed by a third party without divulging the content.

### Step-by-step procedure

Before going into the program, can we discuss the necessary steps to do this encryption?

1. Open the image we want to encrypt
2. (Optional) Converts the image to grayscale
3. Use loops to iterate over the pixels to encrypt them one by one
4. Save the encrypted image to disk

Sounds simple?
Let's do it!

### A Sample Secret Image

Your hands-on directory has a secret image that we have to encrypt before sharing to the world.
The secret is located here: `~/CItraining/module-crypt/images/lion14graytiny.jpg`.
This image is very tiny (39x26 pixels) for a good reason that will be obvious later.
(An alternative is given, `lion14graytiny-x8.png`, which enlarges the picture by 8x in each linear dimension.)

> In case you are curious what the secret image looks like, you can view `lion14graytiny.jpg` or `lion14graytiny-x8.png` using Jupyter notebook by copying the following code snippets and executing them here:
>
> ```python
> from matplotlib import pyplot
> img = pyplot.imread('images/lion14graytiny-x8.png')
> pyplot.imshow(img, cmap='gray', vmin=0, vmax=1.0)
> # if you read the original JPG file, change to: `vmax=255`
> ```

### A Sample Solution: `encrypt_img.py`

> *HINT:* For those liking a challenge, you are strongly encouraged to try making the program yourself before looking at the solution!

A solution script is given in your hands-on package in `~/CItraining/module-crypt/phe_image/encrypt_img.py`.
This script uses only 1 thread (i.e. serial processing) to encrypt the input image, pixel by pixel.
Images that are in color will be converted to grayscale before processing.

This program needs two inputs:

* the input image to be given as the first command-line argument of the program

* the Paillier public key (prepared beforehand) as a file named `phe_key.pub` in the current directory

The output of the program will be a file named `INPUTFILE.pcrypt`, where `INPUTFILE` is the input filename (including the extension).

From a UNIX shell, run this script with an image file as its first argument, for example:

```bash
python3 encrypt_img.py lion14graytiny.jpg
```

> *HINTS:*
>
> In Wahab HPC, make sure that you have loaded the `DeapSECURE` module.
>
> * In the testing phase, choose very small image to save your waiting time! Say, at most 64x64, because the code will take a lot of time to encrypt!
> * You can also run this command from Jupyter by adding `!` before the command name.
> * You can upload your own image if you like. Additional images with different sizes are available in `~/CItraining/module-crypt/images`.

**EXERCISES**:

* Run the program with `lion14graytiny.jpg` as its input file, if you haven't already.
  Notice how much time it takes.

* Read & understand the `encrypt_img.py` program.

### Scaling Up

**EXERCISES**:

1) Estimate how much time will be required to encrypt larger images (e.g. 180x140, 385x256, or 640x480 grayscale image).

2) If the image has color (RGB), the timing will triple!

**COMMENT**: Given the timing of the crypto operations on Wahab HPC, it looks that by the time the image has >100k pixels, the pain of waiting the serial program to finish encrypting becomes excruciating.

## Decrypting an Encrypted Image (A Coding Exercise)

**OPTIONAL EXERCISE**:
Based on `encrypt_img.py`, try to create the reverse program to decrypt the encrypted image.
Run this program against the output

## Parallelizing Image Encryption Process

If you have done the "Scaling Up" exercises above, you will realize that it does not take too big an input image that the original encryption program will exhaust our patience.
We do need to employ parallel programming to allow this program to complete sooner for larger images.
The next step in our workshop will be to parallelize the image encryption script, so we can leverage multiple cores to shorten the encryption time!
We will do this in the last session of this workshop, stay tuned!